In [68]:
import numpy as np
import cvxpy as cp
import csv
import pandas as pd

In [69]:
data_path = '5v5 - RanksData.csv'
df = pd.read_csv(data_path)
df.dropna(inplace=True)
df.head()
# df

,bnet,discord,Tank SR,DPS SR,Support SR,Wants Tank,Wants Dps,Wants Support
0,Sushi,Salmonnigirii,2.50,2.50,2.50,-1.0,-1.0,1.0
1,Moogle,Moogle,2.50,3.00,3.00,-1.0,0.0,1.0
2,Kaz,Kaz,7.50,3.00,3.00,1.0,0.0,0.0
3,Eternal,Eternal,6.50,3.00,3.00,1.0,-1.0,-1.0
4,Fry,Fry,4.25,5.25,5.25,0.0,1.0,0.0


In [70]:
role_srs = df[['Tank SR', 'DPS SR', 'Support SR']].to_numpy()# * 4
role_comfs = df[['Wants Tank', 'Wants Dps', 'Wants Support']].to_numpy()
df.rename(columns={'bnet': 'Overwatch', 'discord': 'Discord'}, inplace=True)
ids_df = df[['Overwatch', 'Discord']]
# ids_df.head()
# role_srs

In [71]:
n_teams = 2
n_roles = 3
team_size = 5
n = n_teams * team_size
n_players = len(ids_df)


max_sr_diff = 20
avg_sr_weight = 2/5000

id_n = n_teams * n_roles
identity = cp.Variable((n_players, id_n), boolean=True)
constraints = [cp.sum(identity, axis=1) <= 1] # could have someone not play

team_roles = cp.sum(identity, axis=0) # n_teams x n_roles
for i in range(n_teams):
    constraints += [team_roles[3*i] == 1]
    constraints += [team_roles[3*i+1] == 2]
    constraints += [team_roles[3*i+2] == 2]
# constraints += [team_roles == 2]

is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

team_total_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles],role_srs)) for i in range(0,id_n,n_roles)])
team_avg_srs = team_total_srs / team_size
avg_sr = cp.sum(team_avg_srs) / n_teams
sr_deviations = team_avg_srs - avg_sr
constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]
# print("team total srs", team_total_srs.shape)
# print("team avg srs", team_avg_srs.shape)
# print("avg sr", avg_sr.shape)
# print("sr deviations", sr_deviations.shape)

duos = [[8,9], [15, 18], [21, 3]]
is_team = cp.vstack([sum(identity[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
team_assignments = {i: duos[i] for i in range(len(duos))}
for i in range(len(duos)):
    constraints += [is_team[player, i%n_teams] == 1 for player in duos[i]]

cost = cp.sum(comfs) # maximize comfortability
# cost += avg_sr_weight * avg_sr #
cost -= 1/200 * (cp.max(team_avg_srs) - cp.min(team_avg_srs))
# cost += affinity
# cost
objective = cp.Maximize(cost)

prob = cp.Problem(objective, constraints)
prob.solve()

ids = identity.value
is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
teams = np.where(np.all(is_team == 0, axis=1), -1, np.argmax(is_team, axis=1))
roles = np.where(np.all(is_role == 0, axis=1), -1, np.argmax(is_role, axis=1))

print(teams, roles)
srs = np.squeeze(np.take_along_axis(role_srs, roles[:,None], axis=1))
comfs = np.squeeze(np.take_along_axis(role_comfs, roles[:,None], axis=1))
print(srs, comfs)
avg_srs = [np.mean(srs[teams==i]) for i in range(n_teams)]
devs = np.abs(avg_srs - np.mean(srs))
print(np.abs(avg_srs - np.mean(srs)))
print(f"comf: {np.mean(comfs)}, sr: {np.mean(avg_srs)} +- {np.mean(devs)}")
print(np.max(avg_srs), np.min(avg_srs))
print(np.std(avg_srs))

[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 2 1 2 2 0 0 0 1 0 0 1 2 0 1 0 0 0 0 0]
[ 2.5   2.5   7.5   6.5   4.25  8.    7.    2.5   8.    4.5   2.5   2.5
  2.5   2.5  10.    7.5   7.    3.    8.    2.5   2.5   1.5   2.5   6.
  3.5   2.5   5.  ] [-1. -1.  1.  1.  0.  1.  0. -1.  1.  1.  1.  1. -1.  1.  1.  1.  0.  0.
  1.  1. -1.  1. -1.  0.  1. -1.  0.]
[0.00462963 0.02037037]
comf: 0.25925925925925924, sr: 4.6125 +- 0.012500000000000178
4.625 4.6
0.012500000000000178


In [52]:
srs_file = "srs_header.csv"
comfs_file = "comfs_header.csv"
srs_file = "Fall In-House Tournament (Responses) - SRs.csv"
comfs_file = "Fall In-House Tournament (Responses) - Comfs.csv"
ids_file = "Fall In-House Tournament (Responses) - IDs.csv"
has_header = True
skiprows = 1 if has_header else 0

role_srs = np.loadtxt(open(srs_file, "rb"), delimiter=",", skiprows=skiprows)
role_comfs = np.loadtxt(open(comfs_file, "rb"), delimiter=",", skiprows=skiprows)
# IDs = np.loadtxt(open(comfs_file, "rb"), delimiter=",", skiprows=0)
ids_df = pd.read_csv(ids_file)

print("srs", role_srs.shape, np.max(role_srs), np.min(role_srs))
print("comfs", role_comfs.shape, np.max(role_comfs), np.min(role_comfs))
# role_comfs
# role_srs = np.random.random((n, 3))*5000
# # role_comfs = np.random.randint(-1, 2, (n,3))
# print("srs", role_srs.shape, np.max(role_srs), np.min(role_srs))
# print("comfs", role_comfs.shape, np.max(role_comfs), np.min(role_comfs))

srs (24, 3) 4200.0 0.0
comfs (24, 3) 1.0 -1.0


In [53]:
n_teams = 4
n_roles = 3
team_size = 6
n = n_teams * team_size

max_sr_diff = 20
avg_sr_weight = 2/5000

id_n = n_teams * n_roles
identity = cp.Variable((n, id_n), boolean=True)
constraints = [cp.sum(identity, axis=1) == 1]

team_roles = cp.sum(identity, axis=0)
constraints += [team_roles == 2]

is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
print(role_comfs.shape, is_role.shape)
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

team_total_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles],role_srs)) for i in range(0,id_n,n_roles)])
team_avg_srs = team_total_srs / team_size
avg_sr = cp.sum(team_avg_srs) / n_teams
sr_deviations = team_avg_srs - avg_sr
constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

duos = [[8,9], [15, 18], [21, 3]]
is_team = cp.vstack([sum(identity[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
team_assignments = {i: duos[i] for i in range(len(duos))}
for i in range(len(duos)):
    constraints += [is_team[player, i%n_teams] == 1 for player in duos[i]]

cost = cp.sum(comfs)
cost += avg_sr_weight * avg_sr
# cost -= 1/200 * (cp.max(team_avg_srs) - cp.min(team_avg_srs))
# cost += affinity
# cost
objective = cp.Maximize(cost)


print(np.array(constraints).shape)
print(np.array(constraints))
print("COST")
print(cost)
prob = cp.Problem(objective, constraints)
prob.solve()

ids = identity.value
is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
teams = np.argmax(is_team, axis=1)
roles = np.argmax(is_role, axis=1)

print(teams, roles)
srs = np.squeeze(np.take_along_axis(role_srs, roles[:,None], axis=1))
comfs = np.squeeze(np.take_along_axis(role_comfs, roles[:,None], axis=1))
print(srs, comfs)
avg_srs = [np.mean(srs[teams==i]) for i in range(n_teams)]
devs = np.abs(avg_srs - np.mean(srs))
print(np.abs(avg_srs - np.mean(srs)))
print(f"comf: {np.mean(comfs)}, sr: {np.mean(avg_srs)} +- {np.mean(devs)}")
print(np.max(avg_srs), np.min(avg_srs))
print(np.std(avg_srs))

(24, 3) (24, 3)
(11,)
[Equality(Expression(AFFINE, NONNEGATIVE, (np.int32(24),)), Constant(CONSTANT, NONNEGATIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, (np.int32(12),)), Constant(CONSTANT, NONNEGATIVE, ()))
 Inequality(Constant(CONSTANT, ZERO, ()))
 Inequality(Expression(AFFINE, UNKNOWN, (4,)))
 Inequality(Constant(CONSTANT, NONPOSITIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ()))]
COST
Sum(Sum([[1.00 0.00 -1.00]
 [-1.00 -1.00 1.00]
 ...
 [-1.00 1.00 1.00]
 [1.00 0.00 -1.00]] @ Vstack(var4905[0:24, 0] + var4905[0:24, 3] 

In [45]:
df = pd.DataFrame({
    "Battletag": ids_df["Overwatch"],
    "Discord": ids_df["Discord"],
    "SR": srs,
    "Comf": comfs,
    "Role": [role_names[role] for role in roles],
    "Team": [team_names[team] for team in teams]
}).sort_values(["Team", "Role"])
print(df.groupby("Team")["SR"].mean())
# print(df.groupby("Team")["Comf"].mean())
df

# df

Team
Air      3260.000000
Earth    3276.666667
Fire     3292.166667
Water    3283.333333
Name: SR, dtype: float64


,Battletag,Discord,SR,Comf,Role,Team
4,nomad#12895,lauren#1562,3230.0,1.0,DPS,Air
19,splotchylime#1868,splotchylime#6100,3700.0,1.0,DPS,Air
7,CrayCray#11536,CrayCray#0047,2700.0,1.0,Support,Air
12,Saros#11683,Saros#8918,2930.0,1.0,Support,Air
14,Peter#15693,Peter#2421,4000.0,1.0,Tank,Air
22,owTwo#11669,HeAndre#4970,3000.0,1.0,Tank,Air
5,REALM#11702,ps4homescreenmusic#0518,3900.0,0.0,DPS,Earth
11,godofconquez#1127,godofconquezt#5831,2000.0,1.0,DPS,Earth
15,SĭCKO#1152,akir#6939,3750.0,1.0,Support,Earth
18,spidey#11813,itsben#7214,4000.0,1.0,Support,Earth


In [46]:
df.to_csv("matchings2.csv")

In [48]:
dfx = pd.read_csv("matchings1.csv")
print(dfx.groupby("Team")["SR"].mean())
dfx

Team
Air      3240.000000
Earth    3263.333333
Fire     3300.000000
Water    3308.833333
Name: SR, dtype: float64


,Unnamed: 0,Battletag,Discord,SR,Comf,Role,Team
0,16,Rayquaza#11859,Mr.Bae#7026,3600.0,1.0,DPS,Air
1,19,splotchylime#1868,splotchylime#6100,3700.0,1.0,DPS,Air
2,7,CrayCray#11536,CrayCray#0047,2700.0,1.0,Support,Air
3,12,Saros#11683,Saros#8918,2930.0,1.0,Support,Air
4,14,Peter#15693,Peter#2421,4000.0,1.0,Tank,Air
5,17,Redzigzag#1976,Amar JB#8054,2510.0,1.0,Tank,Air
6,4,nomad#12895,lauren#1562,3230.0,1.0,DPS,Earth
7,11,godofconquez#1127,godofconquezt#5831,2000.0,1.0,DPS,Earth
8,15,SĭCKO#1152,akir#6939,3750.0,1.0,Support,Earth
9,18,spidey#11813,itsben#7214,4000.0,1.0,Support,Earth


In [53]:
dfx["Team"][21] = "Air"
dfx["Role"][21] = "Tank"
dfx["Team"][4] = "Water"
dfx["Role"][4] = "Support"
print(dfx.groupby("Team")["SR"].mean())
dfx

Team
Air      3265.000000
Earth    3263.333333
Fire     3300.000000
Water    3283.833333
Name: SR, dtype: float64


/Users/osherlerner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/osherlerner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/osherlerner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is s

,Unnamed: 0,Battletag,Discord,SR,Comf,Role,Team
0,16,Rayquaza#11859,Mr.Bae#7026,3600.0,1.0,DPS,Air
1,19,splotchylime#1868,splotchylime#6100,3700.0,1.0,DPS,Air
2,7,CrayCray#11536,CrayCray#0047,2700.0,1.0,Support,Air
3,12,Saros#11683,Saros#8918,2930.0,1.0,Support,Air
4,14,Peter#15693,Peter#2421,4000.0,1.0,Support,Water
5,17,Redzigzag#1976,Amar JB#8054,2510.0,1.0,Tank,Air
6,4,nomad#12895,lauren#1562,3230.0,1.0,DPS,Earth
7,11,godofconquez#1127,godofconquezt#5831,2000.0,1.0,DPS,Earth
8,15,SĭCKO#1152,akir#6939,3750.0,1.0,Support,Earth
9,18,spidey#11813,itsben#7214,4000.0,1.0,Support,Earth


In [20]:
ids_df = pd.read_csv(ids_file)
ow_names = ids_df["Overwatch"]
teams_dict = {team_names[i]: np.argwhere(teams == i).squeeze() for i in range(n_teams)}
# ids_df["Overwatch"][0]
teams_dict
for team in teams_dict:
    print("\n\nTeam " + team)
    for player in teams_dict[team]:
        print(f"{ow_names[player]}:\t{role_names[roles[player]]}")
    
#     print(ow_names)



Team Water
Enigma#14632:	Tank
Nef#11417:	DPS
LogiC#12582:	DPS
Koani#11331:	Support
AsianBakun#1400:	Tank
Riskyrez#2429:	Support


Team Earth
stuby123#1516:	Tank
REALM#11702:	DPS
godofconquez#1127:	DPS
SĭCKO#1152:	Support
Redzigzag#1976:	Tank
spidey#11813:	Support


Team Fire
Orange#12290:	Support
madaomax#1890:	Support
Bananabread#11652:	DPS
Peter#15693:	Tank
splotchylime#1868:	DPS
R1ckyy#1919:	Tank


Team Air
MadPigDiseaz#1547:	Tank
nomad#12895:	DPS
CrayCray#11536:	Support
Saros#11683:	Support
Rayquaza#11859:	DPS
owTwo#11669:	Tank


In [15]:
np.argwhere(teams == 1).squeeze()

array([ 2,  5, 11, 15, 17, 18])

In [4]:
team_names = ["Water", "Earth", "Fire", "Air"]
for team in teams:
    print(team_names[team])

Fire
Air
Earth
Fire
Air
Earth
Fire
Air
Water
Water
Water
Earth
Air
Water
Fire
Earth
Air
Earth
Earth
Fire
Water
Fire
Air
Water


In [5]:
role_names = ["Support", "DPS", "Tank"]
for role in roles:
    print(role_names[role])

Support
Tank
Tank
Support
DPS
DPS
DPS
Support
Tank
DPS
DPS
DPS
Support
Support
Tank
Support
DPS
Tank
Support
DPS
Tank
Tank
Tank
Support


In [24]:
# 8,9
# 15,18
# 21,3
n_teams = 4
n_roles = 3
team_size = 6
n = n_teams * team_size

max_sr_diff = 120
avg_sr_weight = 1/5000

id_n = n_teams * n_roles
identity = cp.Variable((n, id_n), boolean=True)
constraints = [cp.sum(identity, axis=1) == 1]

team_roles = cp.sum(identity, axis=0)
constraints += [team_roles == 2]

is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

team_total_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles],role_srs)) for i in range(0,id_n,n_roles)])
team_avg_srs = team_total_srs / team_size
avg_sr = cp.sum(team_avg_srs) / n_teams
sr_deviations = team_avg_srs - avg_sr
constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

In [25]:
duos = [[8,9], [15, 18], [21, 3]]
is_team = cp.vstack([sum(identity[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
team_assignments = {i: duos[i] for i in range(len(duos))}
for i in range(len(duos)):
    constraints += [is_team[player, i%n_teams] == 1 for player in duos[i]]
#     constaints += []
# constraints += [is_team[8, 0] == 1]
# constraints += [is_team[9, 0] == 1]



In [5]:
duos = [[8,9], [15, 18], [21, 3]]
is_team = cp.vstack([sum(identity[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
affinity = 0
for a,b in duos:
    affinity += is_team[a].T @ is_team[b]
semi_duos = [[5, 1]]
for a,b in duos:
    affinity += 0.5*is_team[a].T @ is_team[b]

In [26]:
cost = cp.sum(comfs)
cost += 1/5000 * avg_sr
# cost -= 1/200 * (cp.max(team_avg_srs) - cp.min(team_avg_srs))
# cost += affinity
cost

Expression(AFFINE, UNKNOWN, ())

In [27]:
# objective = cp.Maximize(cp.sum(comfs))# + avg_sr * avg_sr_weight)
objective = cp.Maximize(cost)

prob = cp.Problem(objective, constraints)
prob.solve()

ids = identity.value
is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
teams = np.argmax(is_team, axis=1)
roles = np.argmax(is_role, axis=1)

teams, roles

(array([3, 1, 3, 2, 3, 2, 1, 0, 0, 0, 1, 3, 0, 2, 0, 1, 2, 2, 1, 0, 1, 2,
        3, 3]),
 array([0, 2, 2, 0, 1, 1, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 1, 2, 0, 1, 2, 2,
        2, 0]))

In [21]:
X = cp.Variable((100, 100), PSD=True)

# You can use X anywhere you would use
# a normal CVXPY variable.
obj = cp.Minimize(cp.norm(X) + cp.sum(X))
obj

Minimize(Expression(CONVEX, UNKNOWN, ()))

In [22]:
# obj = cp.Minimize(X[0,0] * X[0,0])
cp.Problem(obj, [cp.sum(X) <= 10000]).solve()

-4.309270147332607e-07

In [ ]:
from cvxopt import solvers, matrix, spdiag, sqrt, div

# def robls(A, b, rho):
#     m, n = A.size
n = n_players * n_teams * n_roles


def F(x=None, z=None):
    if x is None: return 0, matrix(0.0, (n,1))
    y = A*x-b
    w = sqrt(rho + y**2)
    f = sum(w)
    Df = div(y, w).T * A
    if z is None: return f, Df
    H = A.T * spdiag(z[0]*rho*(w**-3)) * A
    return f, Df, H
return solvers.cp(F)['x']

In [51]:
identity.value

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.,

In [21]:
n_teams = 4
n_roles = 3
team_size = 6
n = n_teams * team_size

def make_teams(role_srs, role_comfs, max_sr_diff=200, avg_sr_weight=1/5000):
    id_n = n_teams * n_roles
    identity = cp.Variable((n, id_n), boolean=True)
    constraints = [cp.sum(identity, axis=1) == 1]

    team_roles = cp.sum(identity, axis=0)
    constraints += [team_roles == 2]

    is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
    comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
    constraints += [comfs >= 0]

    team_total_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles],role_srs)) for i in range(0,id_n,n_roles)])
    team_avg_srs = team_total_srs / team_size
    avg_sr = cp.sum(team_avg_srs) / n_teams
    sr_deviations = team_avg_srs - avg_sr
    constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

    objective = cp.Maximize(cp.sum(comfs) + avg_sr * avg_sr_weight)
    prob = cp.Problem(objective, constraints)
    prob.solve()

    ids = identity.value
    is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
    is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
    teams = np.argmax(is_team, axis=1)
    roles = np.argmax(is_role, axis=1)

    return teams, roles

In [28]:
# teams, roles = make_teams(role_srs, role_comfs, max_sr_diff=100)
print(teams, roles)
srs = np.squeeze(np.take_along_axis(role_srs, roles[:,None], axis=1))
comfs = np.squeeze(np.take_along_axis(role_comfs, roles[:,None], axis=1))
print(srs, comfs)
avg_srs = [np.mean(srs[teams==i]) for i in range(n_teams)]
print(np.abs(avg_srs - np.mean(srs)))
print(np.mean(comfs))

[3 1 3 2 3 2 1 0 0 0 1 3 0 2 0 1 2 2 1 0 1 2 3 3] [0 2 2 0 1 1 1 0 2 1 1 1 0 0 2 0 1 2 0 1 2 2 2 0]
[3300. 3900. 3500. 2700. 3230. 3900. 3500. 2700. 3300. 3400. 2703. 2000.
 2930. 4150. 4000. 3750. 3600. 2510. 4000. 3700. 2400. 2500. 3000. 3700.] [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1.]
[ 72.79166667 109.95833333  38.875      143.875     ]
0.875


In [ ]:
max_sr_diff = 200

identity = cp.Variable((n, id_n), boolean=True)
constraints = [0 <= identity, identity <= 1]
constraints += [cp.sum(identity, axis=1)==1]

team_roles = cp.sum(identity, axis=0)
constraints += [team_roles == 2]

is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

team_avg_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles], role_srs)) for i in range(0,id_n,n_roles)]) / team_size
avg_sr = cp.sum(team_avg_srs) / n_teams
sr_deviations = team_avg_srs - avg_sr
constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

total_comf = cp.sum(comfs)
objective = cp.Maximize(total_comf + avg_sr/5000)

prob = cp.Problem(objective, constraints)


In [ ]:
prob.solve()
ids = identity.value
cool_ids = [ids[:,i:i+n_roles] for i in range(0,id_n,n_roles)]
print(ids)
is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
teams = np.argmax(is_team, axis=1)
roles = np.argmax(is_role, axis=1)

srs = np.squeeze(np.take_along_axis(role_srs, roles[:,None], axis=1))
comfs = np.squeeze(np.take_along_axis(role_comfs, roles[:,None], axis=1))
print(srs)
print(comfs)
team_sr_avgs = is_team.T @ srs / team_size

print(teams, roles)
# is_role.shape

## dev tests

In [1]:
import numpy as np
# import cvxpy as cp
import cvxopt as cp

In [44]:
n_teams = 4
team_size = 6
n = n_teams * team_size
max_sr_diff = 200
players = np.arange(n)

sup_srs = np.random.random(n)*5000
dps_srs = np.random.random(n)*5000
tank_srs = np.random.random(n)*5000

sup_comfs = np.random.randint(-1, 2, n)
dps_comfs = np.random.randint(-1, 2, n)
tank_comfs = np.random.randint(-1, 2, n)


In [26]:
is_team.T @ is_role
# is_team.T @ srs

Expression(UNKNOWN, NONNEGATIVE, (4,))

In [131]:
n_teams = 4
team_size = 6
n = n_teams * team_size
max_sr_diff = 600

role_srs = np.random.random((n, 3))*5000
role_comfs = np.random.randint(-1, 2, (n,3))
# role_srs = np.random.random((n, 3))*1
# role_comfs = np.random.randint(0, 2, (n,3))

is_role = cp.Variable((n,3), boolean=True)
is_team = cp.Variable((n,4), boolean=True)
constraints = [0 <= is_role, is_role <= 1]
constraints += [0 <= is_team, is_team <= 1]
constraints += [cp.sum(is_role, axis=1)==1, cp.sum(is_team, axis=1)==1]

# teams = np.split(np.arange(n), n_teams)
# team_roles = cp.vstack([cp.sum(is_role[team],axis=0) for team in teams])
team_roles = is_team.T @ is_role
constraints += [team_roles <= 2]

srs = cp.sum(cp.multiply(role_srs, is_role), axis=1)
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

# team_srs = cp.vstack([srs[team] for team in teams])
avg_team_srs = is_team.T @ srs 
# avg_team_srs = cp.sum(team_srs, axis=1) / team_size
avg_sr = cp.sum(srs) / n
sr_deviations = avg_team_srs - avg_sr
# constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

total_comf = cp.sum(comfs)
prob = cp.Problem(cp.Maximize(total_comf), constraints)

In [132]:
n_teams = 4
team_size = 6
n_roles = 3
n = n_teams * team_size
id_n = n_roles*n_teams
max_sr_diff = 200

# role_srs = np.random.random((n, 3))*1
# role_comfs = np.random.randint(0, 2, (n,3))
role_srs = np.random.random((n, 3))*5000
role_comfs = np.random.randint(-1, 2, (n,3))

role_srs, role_comfs

(array([[1767.61158379, 4772.70532046, 1122.73986245],
        [1551.06238039, 1464.23082217, 3506.83061553],
        [3675.06029041, 2221.59167961, 4564.09248401],
        [3626.23264607, 4439.89616242, 4260.48517732],
        [3862.96506874, 1604.65367331,  684.25494103],
        [3366.45062803, 3501.17244308, 3764.69871857],
        [4053.19732809, 1997.63343621, 1866.01411002],
        [1578.96570524, 2165.4173116 , 4359.439635  ],
        [   9.57240075, 2750.3308948 , 1118.46044217],
        [2868.26030607,  431.10317189,  423.5439329 ],
        [ 949.93259815, 4925.27659445, 3229.14643482],
        [2154.91605602, 1507.83287446, 2904.17685449],
        [2265.99129744, 2311.34204199, 1924.1538453 ],
        [1206.18130636,  422.25581528, 3430.94826101],
        [ 886.24930675, 2127.4759882 , 2936.23784812],
        [ 734.28069288, 2986.79844647, 1810.87250326],
        [1278.01177177, 2742.99552663, 4945.29852018],
        [3842.9317441 , 1426.28231009, 2507.67902278],
        [2

In [145]:
identity = cp.Variable((n, id_n), boolean=True)
constraints = [0 <= identity, identity <= 1]
constraints += [cp.sum(identity, axis=1)==1]

team_roles = cp.sum(identity, axis=0)
constraints += [team_roles == 2]
is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
# is_team = cp.vstack([sum(identity[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

# comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
# team_comfs = [[cp.multiply(identity[:,i:i+n_roles], role_comfs)] for i in range(0,id_n,n_roles)]


team_avg_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles], role_srs)) for i in range(0,id_n,n_roles)]) / team_size
avg_sr = cp.sum(team_avg_srs) / n_teams
sr_deviations = team_avg_srs - avg_sr
constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

total_comf = cp.sum(comfs)
prob = cp.Problem(cp.Maximize(total_comf), constraints)

In [147]:
comfs

Expression(AFFINE, UNKNOWN, (24,))

In [148]:
prob.solve()
ids = identity.value
cool_ids = [ids[:,i:i+n_roles] for i in range(0,id_n,n_roles)]
print(ids)
is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
teams = np.argmax(is_team, axis=1)
roles = np.argmax(is_role, axis=1)

srs = np.squeeze(np.take_along_axis(role_srs, roles[:,None], axis=1))
comfs = np.squeeze(np.take_along_axis(role_comfs, roles[:,None], axis=1))
print(srs)
print(comfs)
team_sr_avgs = is_team.T @ srs / team_size

print(teams, roles)
# is_role.shape
team_sr_avgs

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[1767.61158379 3506.83061553 4564.09248401 4439.89616242 3862.9

array([2650.09345619, 2727.98553555, 2778.07394298, 2712.16275588])

In [128]:
# role_srs[:, roles]

# roles[None,:].shape
# role_srs.shape, roles.shape

array([0.6125841 , 0.06369772, 0.6755326 , 0.18800797, 0.28031301,
       0.34758336, 0.33035316, 0.76873587, 0.60293211, 0.11951562,
       0.12987028, 0.46383253, 0.15223794, 0.51671561, 0.11476262,
       0.70087222, 0.02808799, 0.42615228, 0.85735426, 0.76386838,
       0.03345713, 0.84892563, 0.70678572, 0.31679998])

In [94]:
print(prob.solve())
print("role", is_role.value)
print("team", is_team.value)

164.0
role [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
team [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [42]:
print(team_roles.value)
print(teams[0])
print(is_role.value[teams[0]])

None
[0 1 2 3 4 5]


TypeError: 'NoneType' object is not subscriptable

In [42]:
# np.einsum("mn,mn->m", role_srs, is_role)

# cp.vstack([srs[team] for team in teams])
# avg_team_srs
# - avg_sr

In [17]:
role_srs.shape

(24, 3)

In [1]:
import cvxpy as cp
import numpy as np
# Generate a random problem
np.random.seed(0)
m, n= 40, 25

A = np.random.rand(m, n)
b = np.random.randn(m)
# Construct a CVXPY problem
x = cp.Variable(n, integer=True)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
prob = cp.Problem(objective)
prob.solve()

SolverError: Either candidate conic solvers (['GLPK_MI']) do not support the cones output by the problem (SOC), or there are not enough constraints in the problem.

In [2]:
import numpy as np
import cvxpy as cp

M = np.array([[1,0,0,1], [1,0,0,1], [0,1,1,1], [1,0,0,1], [0,0,1,1], [0,0,1,1]])

selection = cp.Variable(M.shape[1], boolean = True)
ones_vec = np.ones(M.shape[1])

constraints = []
for i in range(len(M)):
    constraints.append(M[i] * selection >= 1)

total_genomes = ones_vec * selection

problem = cp.Problem(cp.Minimize(total_genomes), constraints)

problem.solve()

/Users/osherlerner/anaconda3/lib/python3.7/site-packages/cvxpy/expressions/expression.py:558: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/Users/osherlerner/anaconda3/lib/python3.7/site-packages/cvxpy/expressions/expression.py:558: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserW

1.0

In [52]:
n_teams = 4
n_roles = 3
team_size = 6
n = n_teams * team_size

max_sr_diff = 500
avg_sr_weight = 1/5000

id_n = n_teams * n_roles
identity = cp.Variable((n, id_n), boolean=True)
constraints = [cp.sum(identity, axis=1) == 1]

team_roles = cp.sum(identity, axis=0)
constraints += [team_roles == 2]

is_role = cp.vstack([sum(identity[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
comfs = cp.sum(cp.multiply(role_comfs, is_role), axis=1)
constraints += [comfs >= 0]

team_total_srs = cp.hstack([cp.sum(cp.multiply(identity[:,i:i+n_roles],role_srs)) for i in range(0,id_n,n_roles)])
team_avg_srs = team_total_srs / team_size
avg_sr = cp.sum(team_avg_srs) / n_teams
sr_deviations = team_avg_srs - avg_sr
constraints += [sr_deviations <= max_sr_diff, sr_deviations >= -max_sr_diff]

objective = cp.Maximize(cp.sum(comfs))# + avg_sr * avg_sr_weight)
prob = cp.Problem(objective, constraints)
prob.solve()

ids = identity.value
is_role = np.array([sum(ids[:,i+j] for i in range(0,id_n,n_roles)) for j in range(n_roles)]).T
is_team = np.array([sum(ids[:,i+j] for j in range(n_roles)) for i in range(0,id_n,n_roles)]).T
teams = np.argmax(is_team, axis=1)
roles = np.argmax(is_role, axis=1)

teams, roles

(array([0, 3, 3, 1, 2, 2, 0, 1, 1, 3, 2, 0, 3, 2, 1, 2, 2, 0, 3, 0, 0, 3,
        1, 1]),
 array([0, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 1,
        1, 2]))